In [ ]:
import sys
sys.path.append('/home/fjargsto/AcousticBEM/Python')

from InteriorHelmholtzSolver3D import *
from ExampleBoundaries import Sphere

# Test Problem 1
# Dirichlet boundary condition with phi chose to be the same
# as a point-source at the center of the square.
#
frequency = 100.0 # frequency [Hz]
k = frequencyToWavenumber(frequency)

solver = HelmholtzSolver3D(*(Sphere()))

boundaryCondition = BoundaryCondition(solver.aElement.shape[0])
boundaryCondition.alpha.fill(1.0)
boundaryCondition.beta.fill(0.0)

for i in range(solver.aCenters.shape[0]):
    R = norm(solver.aCenters[i, :])
    boundaryCondition.f[i] = np.exp(1j * k * R) / R

boundaryIncidence = BoundaryIncidence(solver.aElement.shape[0])
boundaryIncidence.phi.fill(0.0)
boundaryIncidence.v.fill(0.0)

exteriorPoints = np.array([[0.0000, 0.0000,  2.0000],
                           [0.0500, 0.0000,  4.0000],
                           [0.1000, 0.0000,  8.0000],
                           [0.0500, 0.1000, -2.0000]], dtype=np.float32)

exteriorIncidentPhi = np.zeros(exteriorPoints.shape[0], dtype=np.complex64)

boundarySolution = solver.solveExteriorBoundary(k, boundaryCondition, boundaryIncidence)
exteriorPhi = solver.solveExterior(boundarySolution, exteriorIncidentPhi, exteriorPoints)
print "\n\nTest Problem 1"
print "==============\n"
print boundarySolution
printInteriorSolution(boundarySolution, exteriorPhi)

# Test Problem 2
# Neumann boundary condition with v chosen to be the same
# as a point-source at the center of the square.
#
boundaryCondition.alpha.fill(0.0)
boundaryCondition.beta.fill(1.0)

for i in range(solver.aCenters.shape[0]):
    R = norm(solver.aCenters[i, :])
    a = solver.aVertex[solver.aElement[i, 0], :]
    b = solver.aVertex[solver.aElement[i, 1], :]
    c = solver.aVertex[solver.aElement[i, 2], :]
    normal = Normal3D(a, b, c)
    dot = np.dot(solver.aCenters[i, :], normal)
    boundaryCondition.f[i] = np.exp(1j * k * R) * dot * (1j * k * R - 1.0) / R**3

boundarySolution = solver.solveExteriorBoundary(k, boundaryCondition, boundaryIncidence)
exteriorPhi = solver.solveExterior(boundarySolution, exteriorIncidentPhi, exteriorPoints)
print "\n\nTest Problem 2"
print "==============\n"
print boundarySolution
printInteriorSolution(boundarySolution, exteriorPhi)

# Test Problem 3
# Neumann boundary condition with v=0 on the full boundary.
# The 400Hz field in this case is generated by a point source
# at (0.5, 0.25)
boundaryCondition.alpha.fill(1.0)
boundaryCondition.beta.fill(0.0)

sourcePoint = np.array([0.0, 0.0, 3.0], dtype=np.float32)
for i in range(solver.aCenters.shape[0]):
    x = solver.aCenters[i, :]
    r = sourcePoint - x
    R = norm(r)
    boundaryIncidence.phi[i] = np.exp(1j * k * R) / (4.0 * np.pi * R)
    boundaryCondition.f[i] = boundaryIncidence.phi[i]
    a = solver.aVertex[solver.aElement[i, 0], :]
    b = solver.aVertex[solver.aElement[i, 1], :]
    c = solver.aVertex[solver.aElement[i, 2], :]
    normal = Normal3D(a, b, c)
    dot = -np.dot(r, normal)
    boundaryIncidence.v[i] = np.exp(1j * k * R) * dot * (1j * k * R - 1.0) / (4.0 * np.pi * R**3)

boundarySolution = solver.solveExteriorBoundary(k, boundaryCondition, boundaryIncidence)

for i in range(exteriorPoints.shape[0]):
    R = norm(exteriorPoints[i, :] - sourcePoint)
    exteriorIncidentPhi[i] = np.exp(1j * k * R) / (4.0 * np.pi * R)
    
exteriorPhi = solver.solveExterior(boundarySolution, exteriorIncidentPhi, exteriorPoints)
print "\n\nTest Problem 3"
print "==============\n"
print boundarySolution
printInteriorSolution(boundarySolution, exteriorPhi)

# Test Problem 4
# Neumann boundary condition with v=0 on the full boundary.
# The 400Hz field in this case is generated by a point source
# at (0.5, 0.25)
boundaryCondition.alpha.fill(0.0)
boundaryCondition.beta.fill(1.0)
boundaryCondition.f.fill(0.0)

sourcePoint = np.array([0.0, 0.0, 3.0], dtype=np.float32)
for i in range(solver.aCenters.shape[0]):
    x = solver.aCenters[i, :]
    r = sourcePoint - x
    R = norm(r)
    boundaryIncidence.phi[i] = np.exp(1j * k * R) / (4.0 * np.pi * R)  
    a = solver.aVertex[solver.aElement[i, 0], :]
    b = solver.aVertex[solver.aElement[i, 1], :]
    c = solver.aVertex[solver.aElement[i, 2], :]
    normal = Normal3D(a, b, c)
    dot = -np.dot(r, normal)
    boundaryIncidence.v[i] = np.exp(1j * k * R) * dot * (1j * k * R - 1.0) / (4.0 * np.pi * R**3)

boundarySolution = solver.solveExteriorBoundary(k, boundaryCondition, boundaryIncidence)

for i in range(exteriorPoints.shape[0]):
    R = norm(exteriorPoints[i, :] - sourcePoint)
    exteriorIncidentPhi[i] = np.exp(1j * k * R) / (4.0 * np.pi * R)
    
exteriorPhi = solver.solveExterior(boundarySolution, exteriorIncidentPhi, exteriorPoints)
print "\n\nTest Problem 4"
print "==============\n"
print boundarySolution
printInteriorSolution(boundarySolution, exteriorPhi)
